# Electricity transformer data preparation

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/raw/electricity_transformer.csv')

In [3]:
DATA = data

A feature that can be relevant is the current hour: 

In [4]:
#what hour is it
DATA['hour'] = pd.to_datetime(DATA.date).dt.hour

We are going to add some additional features related to the target on past times:

In [5]:
#last 24 hours

for i in range(1, 4*24+1):
    DATA['OT_'+str(i)] = DATA['OT'].shift(i)

In [6]:
#mean last hour
mean_last_hour = []

for i in range(len(DATA)):
    value = DATA.loc[i, ['OT_1', 'OT_2', 'OT_3', 'OT_4']].mean()
    mean_last_hour += [value]

    
DATA['OT_mean_last_hour'] = mean_last_hour

In [7]:
#mean last 24 hours

mean_last_day = []

for i in range(len(DATA)):
    value = DATA.loc[i,['OT_'+str(i) for i in range(1, 4*24)]].mean()
    mean_last_day += [value]

    
DATA['OT_mean_last_day'] = mean_last_day

In [8]:
#ratio last checkpoint (if ratio>1, increasing trend)

DATA['OT_ratio_1'] = DATA.OT / DATA.OT_1

In [9]:
#ratio last hour

DATA['OT_ratio_hour'] = DATA.OT / DATA.OT_4

In [10]:
#ratio last day

DATA['OT_ratio_day'] = DATA.OT / DATA.OT_96

Of course, we add the column that we are going to predict that is the Oil Temperature lagged once in the future:

In [11]:
DATA['OT_next'] = DATA['OT'].shift(-1)

In [12]:
DATA.head()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,hour,OT_1,...,OT_93,OT_94,OT_95,OT_96,OT_mean_last_hour,OT_mean_last_day,OT_ratio_1,OT_ratio_hour,OT_ratio_day,OT_next
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.459999
1,2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999,0,30.531000,...,NaN,NaN,NaN,NaN,30.5310,30.5310,0.997674,NaN,NaN,30.038000
2,2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000,0,30.459999,...,NaN,NaN,NaN,NaN,30.4955,30.4955,0.986146,NaN,NaN,27.013000
3,2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000,0,30.038000,...,NaN,NaN,NaN,NaN,30.3430,30.3430,0.899294,NaN,NaN,27.787001
4,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001,1,27.013000,...,NaN,NaN,NaN,NaN,29.5105,29.5105,1.028653,0.910124,NaN,27.716999


In [13]:
DATA.tail()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,hour,OT_1,...,OT_93,OT_94,OT_95,OT_96,OT_mean_last_hour,OT_mean_last_day,OT_ratio_1,OT_ratio_hour,OT_ratio_day,OT_next
69675,2018-06-26 18:45:00,9.310,3.550,5.437,1.670,3.868,1.462,9.567,18,9.778,...,10.200,10.271,10.552,10.622,9.83100,9.711516,0.978421,0.971368,0.900678,9.567
69676,2018-06-26 19:00:00,10.114,3.550,6.183,1.564,3.716,1.462,9.567,19,9.567,...,10.060,10.200,10.271,10.552,9.76050,9.701147,1.000000,0.978421,0.906653,9.426
69677,2018-06-26 19:15:00,10.784,3.349,7.000,1.635,3.746,1.432,9.426,19,9.567,...,9.989,10.060,10.200,10.271,9.70775,9.693737,0.985262,0.950297,0.917730,9.426
69678,2018-06-26 19:30:00,11.655,3.617,7.533,1.706,4.173,1.523,9.426,19,9.426,...,9.989,9.989,10.060,10.200,9.58450,9.685589,1.000000,0.964001,0.924118,9.778
69679,2018-06-26 19:45:00,12.994,3.818,8.244,1.777,4.721,1.523,9.778,19,9.426,...,9.849,9.989,9.989,10.060,9.49650,9.678916,1.037344,1.022055,0.971968,NaN


Lastly, for avoiding NaN, we are going to delete the first few rows and the last one:


In [14]:
df = DATA.drop(DATA.index[:96])

In [15]:
df = df.drop(df.index[-1])

In [16]:
df.isna().sum().sum()

617

In [17]:
df.isna().sum()

date                  0
HUFL                  0
HULL                  0
MUFL                  0
MULL                  0
                   ... 
OT_mean_last_day      0
OT_ratio_1          315
OT_ratio_hour       252
OT_ratio_day         50
OT_next               0
Length: 111, dtype: int64

We still having some issues because Oil Temperature is eventually zero and we have calculated ratios

In [18]:
df = df.fillna(-1)

In [19]:
df.isna().sum().sum()

0

And now we separate in train/validation/test (70/15/15 in proportion, respectively):

In [21]:
train_date = df.loc[int(round(len(df)*0.7 + 96))].date

In [23]:
validation_date = df.loc[int(round(len(df)*0.85 + 96))].date

In [27]:
train = df[df.date<train_date]
train.shape

(48708, 111)

In [35]:
validation = df[train_date<=df.date]
validation = validation[validation.date<validation_date]
validation.shape

(10438, 111)

In [29]:
test = df[df.date>=validation_date]
test.shape

(10437, 111)

In [43]:
train.to_parquet('F:/Users/Manuel García Plaza/Desktop/TFG/data/model_input/train_sets/electricity_transformer.parquet')
validation.to_parquet('F:/Users/Manuel García Plaza/Desktop/TFG/data/model_input/validation_sets/electricity_transformer.parquet')
test.to_parquet('F:/Users/Manuel García Plaza/Desktop/TFG/data/model_input/test_sets/electricity_transformer.parquet')